In [1]:
import os
import joblib
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

def load_images_from_subdirectories(base_dir, target_size=(64, 64)):
    """
    Carga imágenes de subdirectorios, asigna etiquetas y preprocesa.
    """
    images = []
    labels = []
    label_names = []
    label_map = {}

    for i, class_name in enumerate(sorted(os.listdir(base_dir))):
        class_dir = os.path.join(base_dir, class_name)
        if os.path.isdir(class_dir):
            # Only add to label_names if it's a valid directory representing a class
            label_names.append(class_name)
            label_map[class_name] = i
            print(f"Cargando imágenes de la clase: {class_name} (etiqueta: {i})")

            for img_name in os.listdir(class_dir):
                img_path = os.path.join(class_dir, img_name)
                if img_name.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
                    try:
                        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                        if img is None:
                            print(f"Advertencia: No se pudo cargar la imagen: {img_path}")
                            continue
                        img = cv2.resize(img, target_size)
                        images.append(img.flatten())
                        labels.append(label_map[class_name])
                    except Exception as e:
                        print(f"Error procesando {img_path}: {e}")

    X = np.array(images)
    y = np.array(labels)

    if X.size == 0:
        raise ValueError(f"No se cargaron imágenes del directorio {base_dir}. Verifica las rutas y los tipos de archivo.")

    print(f"\nTotal de imágenes cargadas: {len(images)}")
    print(f"Forma de X (características): {X.shape}")
    print(f"Forma de y (etiquetas): {y.shape}")
    print(f"Clases detectadas: {label_names}")

    return X, y, label_names

# Ruta a tu dataset
image_dir = 'dataset_imagenes'
image_target_size = (64, 64)

try:
    X, y, label_names = load_images_from_subdirectories(image_dir, target_size=image_target_size)
except ValueError as e:
    print(f"Error al cargar imágenes: {e}")
    exit()

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

n_components = min(50, X_train.shape[0] - 1, X_train.shape[1] - 1)
if n_components <= 0:
    print(f"Error: n_components para PCA es {n_components}. Asegúrate de tener suficientes imágenes y características.")
    exit()

pca = PCA(n_components=n_components)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

print(f"\nForma de X_train_pca después de PCA: {X_train_pca.shape}")
print("\nAutovectores (componentes principales):")
print(pca.components_)
print("\nAutovalores (varianza explicada por componente):")
print(pca.explained_variance_)
print("\Varianza explicada acumulada:")
print(np.sum(pca.explained_variance_ratio_))

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_pca, y_train)

y_pred = knn.predict(X_test_pca)
accuracy = accuracy_score(y_test, y_pred)
print(f"\nPrecisión del clasificador K-NN en el conjunto de prueba: {accuracy:.2f}")

print("\nReporte de clasificación:")


# Obtener las etiquetas únicas presentes en el conjunto de prueba
unique_labels_in_test = np.unique(np.concatenate((y_test, y_pred)))
# Mapear estas etiquetas numéricas a sus nombres correspondientes
target_names_for_report = [label_names[i] for i in unique_labels_in_test]

print(classification_report(y_test, y_pred, labels=unique_labels_in_test, target_names=target_names_for_report, zero_division='warn'))
# --- FIN DEL CAMBIO ---

model_dir = 'saved_models'
os.makedirs(model_dir, exist_ok=True)

joblib.dump(scaler, os.path.join(model_dir, 'scaler.pkl'))
joblib.dump(pca, os.path.join(model_dir, 'pca.pkl'))
joblib.dump(knn, os.path.join(model_dir, 'knn_model.pkl'))
joblib.dump(label_names, os.path.join(model_dir, 'label_names.pkl'))
joblib.dump(image_target_size, os.path.join(model_dir, 'image_target_size.pkl'))

print(f"\nModelos y parámetros guardados en el directorio: {model_dir}")

Cargando imágenes de la clase: .ipynb_checkpoints (etiqueta: 0)
Cargando imágenes de la clase: entretenimiento (etiqueta: 1)


libpng warning: iCCP: known incorrect sRGB profile


Cargando imágenes de la clase: familia (etiqueta: 2)


libpng warning: iCCP: known incorrect sRGB profile


Cargando imágenes de la clase: trabajo (etiqueta: 3)

Total de imágenes cargadas: 1931
Forma de X (características): (1931, 4096)
Forma de y (etiquetas): (1931,)
Clases detectadas: ['.ipynb_checkpoints', 'entretenimiento', 'familia', 'trabajo']

Forma de X_train_pca después de PCA: (1544, 50)

Autovectores (componentes principales):
[[ 0.01772166  0.01792789  0.01774637 ...  0.0161627   0.01608458
   0.01597658]
 [-0.02074647 -0.02058094 -0.02028771 ...  0.01640109  0.01479615
   0.01337038]
 [ 0.02152087  0.0205977   0.02162634 ...  0.03658506  0.03639779
   0.03603309]
 ...
 [-0.00707094 -0.01499981 -0.01247387 ...  0.01150783  0.01290523
   0.0151962 ]
 [-0.01450359 -0.01262341 -0.00967475 ... -0.00835813 -0.00938206
  -0.01537317]
 [-0.00899951 -0.0076628  -0.00815748 ... -0.01422401 -0.01390937
  -0.0189481 ]]

Autovalores (varianza explicada por componente):
[1953.89153675  166.02081739   97.76089356   71.40799057   62.42360576
   47.37389958   37.6201964    30.40548064   27.0997